In [1]:
!pip install dlt[duckdb]

In [2]:
!dlt --version

dlt 1.6.1


In [3]:
!pip install --upgrade packaging

In [4]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator

BASE_API_URL = "https://us-central1-dlthub-analytics.cloudfunctions.net"

@dlt.resource(name="ny_taxi_data")
def ny_taxi():
    client = RESTClient(
        base_url = BASE_API_URL,
        paginator = PageNumberPaginator(
            base_page=1,
            total_path=None
        )
    )
    
    for page in client.paginate("data_engineering_zoomcamp_api"):
        yield page

pipeline = dlt.pipeline(
    pipeline_name="ny_taxi_pipeline",
    destination="duckdb",
    dataset_name="ny_taxi_data"
)

In [6]:
load_info = pipeline.run(ny_taxi)
print(load_info)

Pipeline ny_taxi_pipeline load step completed in 1.47 seconds
1 load package(s) were loaded to destination duckdb and into dataset ny_taxi_data
The duckdb destination used duckdb:///C:\Users\ipek.guler\Desktop\Data-Engineering-Zoomcamp\03-dlt-workshop\ny_taxi_pipeline.duckdb location to store data
Load package 1739727621.2297852 is LOADED and contains no failed jobs


In [7]:
import duckdb

# A database '<pipeline_name>.duckdb' was created in working directory so just connect to it

# Connect to the DuckDB database
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# Set search path to the dataset
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Describe the dataset
conn.sql("DESCRIBE").df()


database        schema                 name  \
0  ny_taxi_pipeline  ny_taxi_data           _dlt_loads   
1  ny_taxi_pipeline  ny_taxi_data  _dlt_pipeline_state   
2  ny_taxi_pipeline  ny_taxi_data         _dlt_version   
3  ny_taxi_pipeline  ny_taxi_data         ny_taxi_data   

                                        column_names  \
0  [load_id, schema_name, status, inserted_at, sc...   
1  [version, engine_version, pipeline_name, state...   
2  [version, engine_version, inserted_at, schema_...   
3  [end_lat, end_lon, fare_amt, passenger_count, ...   

                                        column_types  temporary  
0  [VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...      False  
1  [BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...      False  
2  [BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...      False  
3  [DOUBLE, DOUBLE, DOUBLE, BIGINT, VARCHAR, DOUB...      False

In [8]:
df = pipeline.dataset(dataset_type="default").ny_taxi_data.df()
df

end_lat    end_lon  fare_amt  passenger_count payment_type  \
0      40.742963 -73.980072      45.0                1       Credit   
1      40.740187 -74.005698       6.5                1       Credit   
2      40.718043 -74.004745      12.5                5       Credit   
3      40.739637 -73.985233       4.9                1         CASH   
4      40.730032 -73.852693      25.7                1         CASH   
...          ...        ...       ...              ...          ...   
19995  40.783522 -73.970690       5.7                1         CASH   
19996  40.777200 -73.964197       4.1                1         CASH   
19997  40.780172 -73.957617       6.1                1         CASH   
19998  40.777342 -73.957242       5.7                1         CASH   
19999  40.757122 -73.986293       6.5                1       Credit   

       start_lat  start_lon  tip_amt  tolls_amt  total_amt  trip_distance  \
0      40.641525 -73.787442      9.0       4.15      58.15          17.52   
1      40.722065 -74.009767      1.0       0.00       8.50           1.56   
2      40.761945 -73.983038      2.0       0.00      15.50           3.37   
3      40.749802 -73.992247      0.0       0.00       5.40           1.11   
4      40.776825 -73.949233      0.0       4.15      29.85          11.09   
...          ...        ...      ...        ...        ...            ...   
19995  40.778560 -73.953660      0.0       0.00       5.70           1.16   
19996  40.779800 -73.974297      0.0       0.00       4.10           0.89   
19997  40.788388 -73.976758      0.0       0.00       6.10           1.30   
19998  40.773828 -73.956690      0.0       0.00       6.20           0.97   
19999  40.756313 -73.972948      2.0       0.00       8.50           0.92   

         trip_dropoff_date_time     trip_pickup_date_time  surcharge  \
0     2009-06-14 23:48:00+00:00 2009-06-14 23:23:00+00:00        0.0   
1     2009-06-18 17:43:00+00:00 2009-06-18 17:35:00+00:00        1.0   
2     2009-06-10 18:27:00+00:00 2009-06-10 18:08:00+00:00        1.0   
3     2009-06-14 23:58:00+00:00 2009-06-14 23:54:00+00:00        0.5   
4     2009-06-13 13:23:00+00:00 2009-06-13 13:01:00+00:00        0.0   
...                         ...                       ...        ...   
19995 2009-06-19 11:28:00+00:00 2009-06-19 11:22:00+00:00        0.0   
19996 2009-06-17 07:43:00+00:00 2009-06-17 07:41:00+00:00        0.0   
19997 2009-06-19 11:46:00+00:00 2009-06-19 11:39:00+00:00        0.0   
19998 2009-06-17 04:19:00+00:00 2009-06-17 04:13:00+00:00        0.5   
19999 2009-06-17 08:34:00+00:00 2009-06-17 08:24:00+00:00        0.0   

      vendor_name        _dlt_load_id         _dlt_id  store_and_forward  
0             VTS   1739726376.937028  W+YVDJOowh13Xw                NaN  
1             VTS   1739726376.937028  iQaIXkdrYKDXdA                NaN  
2             VTS   1739726376.937028  gKcFP5TKxYmU1A                NaN  
3             VTS   1739726376.937028  /zUeX7bKS3SMig                NaN  
4             VTS   1739726376.937028  sADy91IV7V/eUA                NaN  
...           ...                 ...             ...                ...  
19995         VTS  1739727621.2297852  TCNOzpS5Kv7f/g                NaN  
19996         VTS  1739727621.2297852  7E6C6DkV10L7Xw                NaN  
19997         VTS  1739727621.2297852  dU5vjhP5ngdRyQ                NaN  
19998         VTS  1739727621.2297852  S1PfowGO5jSKwA                NaN  
19999         VTS  1739727621.2297852  K8SL2QB7v5fZ0A                NaN  

[20000 rows x 18 columns]

In [9]:
with pipeline.sql_client() as client:
    res = client.execute_sql(
            """
            SELECT
            AVG(date_diff('minute', trip_pickup_date_time, trip_dropoff_date_time))
            FROM ny_taxi_data;
            """
        )
    # Prints column values of the first row
    print(res)

[(12.3049,)]
